In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torchvision.datasets as dset
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchvision.models as models
import cv2
import sys
import math
import random
import splitfolders
import torchsummary
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

In [2]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [3]:
train_df['img_path'] = './train_mfcc/' + train_df['id'] + '.jpeg'
test_df['img_path'] = './test_mfcc/' + test_df['id'] + '.jpeg'

In [4]:
train_df

,id,path,label,img_path
0,TRAIN_0000,./train/TRAIN_0000.wav,1,./train_mfcc/TRAIN_0000.jpeg
1,TRAIN_0001,./train/TRAIN_0001.wav,2,./train_mfcc/TRAIN_0001.jpeg
2,TRAIN_0002,./train/TRAIN_0002.wav,4,./train_mfcc/TRAIN_0002.jpeg
3,TRAIN_0003,./train/TRAIN_0003.wav,5,./train_mfcc/TRAIN_0003.jpeg
4,TRAIN_0004,./train/TRAIN_0004.wav,4,./train_mfcc/TRAIN_0004.jpeg
...,...,...,...,...
4996,TRAIN_4996,./train/TRAIN_4996.wav,5,./train_mfcc/TRAIN_4996.jpeg
4997,TRAIN_4997,./train/TRAIN_4997.wav,0,./train_mfcc/TRAIN_4997.jpeg
4998,TRAIN_4998,./train/TRAIN_4998.wav,1,./train_mfcc/TRAIN_4998.jpeg
4999,TRAIN_4999,./train/TRAIN_4999.wav,1,./train_mfcc/TRAIN_4999.jpeg


In [5]:
test_df

,id,path,img_path
0,TEST_0000,./test/TEST_0000.wav,./test_mfcc/TEST_0000.jpeg
1,TEST_0001,./test/TEST_0001.wav,./test_mfcc/TEST_0001.jpeg
2,TEST_0002,./test/TEST_0002.wav,./test_mfcc/TEST_0002.jpeg
3,TEST_0003,./test/TEST_0003.wav,./test_mfcc/TEST_0003.jpeg
4,TEST_0004,./test/TEST_0004.wav,./test_mfcc/TEST_0004.jpeg
...,...,...,...
1876,TEST_1876,./test/TEST_1876.wav,./test_mfcc/TEST_1876.jpeg
1877,TEST_1877,./test/TEST_1877.wav,./test_mfcc/TEST_1877.jpeg
1878,TEST_1878,./test/TEST_1878.wav,./test_mfcc/TEST_1878.jpeg
1879,TEST_1879,./test/TEST_1879.wav,./test_mfcc/TEST_1879.jpeg


In [6]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [7]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':1000,
    'LEARNING_RATE':1e-3,
    'BATCH_SIZE':32,
    'SEED':42
}

In [8]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

In [9]:
train, val, _, _ = train_test_split(train_df, train_df['label'], test_size=0.1, random_state=CFG['SEED'], stratify=train_df['label'])

In [10]:
train['label'].value_counts()

0    780
2    773
3    767
5    767
1    763
4    650
Name: label, dtype: int64

In [11]:
val['label'].value_counts()

0    87
2    86
5    86
1    85
3    85
4    72
Name: label, dtype: int64

In [12]:
tfms = A.Compose([
    A.Resize(width=CFG['IMG_SIZE'], height=CFG['IMG_SIZE']),
    A.Normalize()
], p=1)

In [13]:
class Dataset(Dataset):
    def __init__(self, df, path_list, label_list, tfms=None):
        self.df = df
        self.path_list = path_list
        self.label_list = label_list
        self.tfms = tfms
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = cv2.imread(self.path_list[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        image = self.tfms(image=img)['image']
        image = torch.tensor(np.array(image)).permute(2, 0, 1)
        
        if self.label_list is not None:
            label = self.label_list[idx]
            return image, label
        else:
            return image

In [14]:
train_dataset = Dataset(df=train, path_list=train['img_path'].values, label_list=train['label'].values, tfms=tfms)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = Dataset(df=val,path_list=val['img_path'].values, label_list=val['label'].values, tfms=tfms)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

in_planes = 64
num_classes = 6

def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(
        in_planes,
        out_planes,
        kernel_size=3,
        stride=stride,
        bias=False,
        padding = 1,
        padding_mode='zeros'
    )


class Residual(nn.Module):
    def __init__(self, numIn, numOut, stride = 1):
        super(Residual, self).__init__()
        self.numIn = numIn
        self.numOut = numOut
        self.cab = CAB(numOut)
        self.stride = stride
        self.conv1 = nn.Conv2d(self.numIn, self.numOut, bias = False, kernel_size = 3,stride = self.stride,padding = 1)
        self.bn1 = nn.BatchNorm2d(self.numOut)
        self.relu = nn.ReLU(inplace = True)
        self.conv2 = nn.Conv2d(self.numOut, self.numOut, bias = False, kernel_size = 3, stride = self.stride, padding = 1)
        self.bn2 = nn.BatchNorm2d(self.numOut)
        
        if self.numIn != self.numOut:
            self.conv4 = nn.Conv2d(self.numIn, self.numOut, bias = True, kernel_size = 1)
            
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        
        if self.numIn != self.numOut:
            residual = self.conv4(x)
        
        return out + residual

    

class ChannelAttention(nn.Module):
    def __init__(self, channel):
        super(ChannelAttention, self).__init__()
        self.attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(channel, channel, 1, padding=0, bias=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(channel, channel, 1, padding=0, bias=True),
            nn.Sigmoid())
    def forward(self, x):
        y = self.attention(x)
        return x * y
    
    
class CAB(nn.Module):
    def __init__(self, channel):
        super(CAB, self).__init__()
        self.cab = nn.Sequential(
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, stride=1),
            nn.GELU(),
            nn.Conv2d(channel, channel, kernel_size=3, padding=1, stride=1),
            ChannelAttention(channel)
            )
    def forward(self, x):
        return self.cab(x)
    
    
class ResNet(nn.Module):
    def __init__(self, in_planes, num_classes):
        super(ResNet, self).__init__()
        self.in_planes = in_planes
        self.conv = nn.Conv2d(3, self.in_planes, kernel_size = 3, stride = 1, padding = 3, padding_mode='zeros', bias=False)
        self.conv_g = nn.Conv2d(2, self.in_planes, kernel_size = 3, stride = 1, padding = 1, padding_mode='zeros', bias=False)
        self.cab = CAB(in_planes)
        self.bn = nn.BatchNorm2d(self.in_planes)
        self.relu = nn.ReLU(inplace = True)
        self.res1 = Residual(in_planes,in_planes)
        self.res2 = Residual(in_planes,in_planes)
        self.res3 = Residual(in_planes,in_planes)
        self.res4 = Residual(in_planes,in_planes)
        self.res5 = Residual(in_planes,in_planes)
        self.res6 = Residual(in_planes,in_planes)
        self.res7 = Residual(in_planes,in_planes)
        self.res8 = Residual(in_planes,in_planes)
        self.res7 = Residual(in_planes,in_planes)
        self.res8 = Residual(in_planes,in_planes)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.avgpool = nn.AvgPool2d(2, stride=2)
        self.gap = nn.AdaptiveAvgPool2d((1, 1))
        self.linear  = nn.Linear(in_planes, num_classes)
    
    def forward(self, x):
        out = self.conv(x)
        out = self.cab(out)
        out = self.bn(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = self.res1(out)
        out = self.res2(out)
        out = self.maxpool(out)
        out = self.res3(out)
        out = self.res4(out)
        out = self.maxpool(out)
        out = self.res5(out)
        out = self.res6(out)
        out = self.maxpool(out)
        out = self.gap(out)

        out = torch.flatten(out, 1)
        out = self.linear(out)
        return out

In [16]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    criterion = nn.CrossEntropyLoss().to(device)
    
    best_val_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for images, labels in tqdm(iter(train_loader)):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(images)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')
        
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_val_score < _val_score:
            best_val_score = _val_score
            best_model = model
            
            torch.save(model, 'save_model/' + 'best_0515.pth'.format(epoch))

    
    return best_model

In [17]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, trues = [], []
    
    with torch.no_grad():
        for images, labels in tqdm(iter(val_loader)):
            images = images.to(device)
            labels = labels.to(device)
            
            logit = model(images)
            
            loss = criterion(logit, labels)
            
            val_loss.append(loss.item())
            
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
            trues += labels.detach().cpu().numpy().tolist()
        
        _val_loss = np.mean(val_loss)
    
    _val_score = f1_score(trues, preds, average='macro')
    return _val_loss, _val_score

In [18]:
model = ResNet(64, 6)
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

100%|████████████████████████████████████████████| 16/16 [00:01<00:00,  8.63it/s]


Epoch [1], Train Loss : [1.68053] Val Loss : [14.05576] Val F1 : [0.04884]


100%|████████████████████████████████████████████| 16/16 [00:01<00:00, 10.87it/s]


Epoch [2], Train Loss : [1.56853] Val Loss : [4.23964] Val F1 : [0.11684]


100%|████████████████████████████████████████████| 16/16 [00:01<00:00, 10.83it/s]


...

Epoch [998], Train Loss : [1.08245] Val Loss : [1.14104] Val F1 : [0.55716]


100%|████████████████████████████████████████████| 16/16 [00:01<00:00, 10.61it/s]


Epoch [999], Train Loss : [1.08686] Val Loss : [1.14223] Val F1 : [0.56287]


100%|████████████████████████████████████████████| 16/16 [00:01<00:00, 10.88it/s]

Epoch [1000], Train Loss : [1.08608] Val Loss : [1.14338] Val F1 : [0.55933]


In [21]:
test_dataset = Dataset(df=test_df,path_list=test_df['img_path'].values, label_list=None, tfms=tfms)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [22]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for img in tqdm(iter(test_loader)):
            img = img.to(device)
            logit = model(img)
            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [23]:
preds = inference(model, test_loader, device)

100%|████████████████████████████████████████████| 59/59 [00:06<00:00,  9.79it/s]


In [25]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = preds
submission.to_csv('./mfcc_0515_submission.csv', index=False)